In [22]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=100
TARGET_DIR='/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0100/scfs'
TARGET_SPECIES='Cl'
####################################################


os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()





def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()


av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")





    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f1
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(TARGET_DIR))
print("==================================")
print("""
target_atom_labels:\t{}\n
av_theta_xs:\t{}\n    
av_theta_ys:\t{}\n  
av_theta_y2s:\t{}\n
av_theta_x2s:\t{}\n 
cq_coef0s =:\t{}\n
cq_coef1s:\t{}\n
eta_coefs:\t{}\n
cq0s:\t{}\n
cqprime0s:\t{}\n
cqprime1s:\t{}\n 
eta0s:\t{}\n
etaprimes:\t{}\n
fq0s:\t{}\n 
fqprimes:\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()





endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()





Begin on Wed Jan 17 19:03:57 2018

Working dir:	/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0100/scfs


TEMP:            100
TARGET_DIR:      /Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0100/scfs
TARGET_SPECIES:  Cl


There are 1174 efg files in this directory.

114 files were skipped.
between efg.590.out and efg.1173.out
display variable "skipped_files" to see all files not opened.
1060 efg files opened without error.

The indices of Cl atoms are:
1 12 18 24 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
900 
Report for atomic specie:	1
Working Dir:	/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0100/scfs
<tx>:	 -0.00938197685448 	<ty>:	 0.0406740456305 
 <tx^2>	: 0.00139965560335 	<ty^2>: 0.00353982705654 

cq0=	-69.2026
eta0=	0.09566
target:	/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0100/scfs
cwd:	/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0100/scfs
T=1

In [3]:
print()

In [23]:

os.listd

['scf.938.out',
 'scf.784.in',
 'scf.415.in',
 'efg.863.in',
 'efg.760.in',
 'efg.1280.out',
 'scf.887.in',
 'scf.1001.in',
 'scf.1141.in',
 'efg.923.in',
 'scf.555.in',
 'efg.832.in',
 'scf.904.out',
 'scf.444.in',
 'scf.1050.in',
 'efg.876.out',
 'efg.731.in',
 'scf-493.efg.magres',
 'efg.862.out',
 'scf.996.in',
 'scf.1110.in',
 'scf.1281.in',
 'scf.910.out',
 'scf.695.in',
 'scf.504.in',
 'scf-1082.efg.magres',
 'efg.972.in',
 'efg.137.out',
 'efg.453.in',
 'scf.825.in',
 'scf.723.out',
 'scf.1332.in',
 '1375 infiles.  @info',
 'scf.726.in',
 'efg.889.out',
 'scf-40.efg.magres',
 'efg.981.in',
 'efg.8.out',
 'scf.666.in',
 'scf.965.in',
 'scf.737.out',
 'scf.1272.in',
 'efg.123.out',
 'efg.513.in',
 'scf.874.in',
 'scf.1363.in',
 'efg.402.in',
 'efg.793.in',
 'scf-1196.efg.magres',
 'scf-77.efg.magres',
 'efg.890.in',
 'scf.777.in',
 'scf-909.efg.magres',
 'scf-587.efg.magres',
 'scf.637.in',
 'efg.542.in',
 'scf.934.in',
 'scf.1223.in',
 'scf.1124.in',
 'efg.1082.out',
 'scf.284.o